Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-data-dependency-steps.png)

# Azure Machine Learning Pipelines with Data Dependency
In this notebook, we will see how we can build a pipeline with implicit data dependency.

### Azure Machine Learning and Pipeline SDK-specific Imports

In [1]:
import azureml.core
import azureml.dataprep
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.data import TabularDataset
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


### Initialize Workspace and Retrieve a Compute Target

In [2]:
ws = Workspace.from_config()
print("== Workspace:")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure blob storage)
# def_blob_store = ws.get_default_datastore()
blob_store = Datastore(ws, "workspaceblobstore")
print("== Datastore: {}".format(blob_store.name))

# list compute targets
print("== Compute targets:")
for ct in ws.compute_targets:
    print("  " + ct)
    
# Retrieve a compute target    
from azureml.core.compute_target import ComputeTargetException
aml_compute_target = "agd-training-cpu"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("== AML compute target attached: " + aml_compute_target)
except ComputeTargetException:
    print("== AML compute target not found: " + aml_compute_target)

== Workspace:
agd-ml-demo
azure-ml-workshop
westus2
c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60
== Datastore: workspaceblobstore
== Compute targets:
  agd-training-cpu
  agd-training-gpu
== AML compute target attached: agd-training-cpu


### Create Compute Configuration

This step uses a docker image, use a [**RunConfiguration**](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfiguration?view=azure-ml-py) to specify these requirements and use when creating the PythonScriptStep. 

In [3]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify dependencies
#run_config.environment.python.conda_dependencies = CondaDependencies.create(
#    conda_packages=['pandas'],
#    pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]', 'azureml-train-automl'], 
#    pin_sdk_version=False)
run_config.environment.python.conda_dependencies = CondaDependencies(
    conda_dependencies_file_path='data-prep-pipeline.yml')

#
print("== Run Configuration created")

== Run Configuration created


## Building Pipeline Steps with Inputs and Outputs
As mentioned earlier, a step in the pipeline can take data as input. This data can be a data source that lives in one of the accessible data locations, or intermediate data produced by a previous step in the pipeline.

### Datasources as Datasets

In [4]:
# hourly time series
h_time_series_1_ds = Dataset.get_by_name(ws,"h_time_series_1")
h_time_series_2_ds = Dataset.get_by_name(ws,"h_time_series_2")
h_time_series_3_ds = Dataset.get_by_name(ws,"h_time_series_3")
# daily time series
d_time_series_1_dr = DataReference(datastore=blob_store,
                                   data_reference_name="d_time_series_1",
                                   path_on_datastore="datasets/time-series/X1.csv")
d_time_series_2_dr = DataReference(datastore=blob_store,
                                   data_reference_name="d_time_series_2",
                                   path_on_datastore="datasets/time-series/X2.csv")

print("== Datasets metadata retrieved")

== Datasets metadata retrieved


### Intermediate/Output Data
Intermediate data (or output of a Step) is represented by **[PipelineData](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinedata?view=azure-ml-py)** object. PipelineData can be produced by one step and consumed in another step by providing the PipelineData object as an output of one step and the input of one or more steps.

In [5]:
# Define intermediate data using PipelineData
# hourly series that need to have timestampts generated from columns and pivoted if necessary
h_time_series_1_pivot_pd = PipelineData("h_time_series_1_pivot",datastore=blob_store)
h_time_series_2_pivot_pd = PipelineData("h_time_series_2_pivot",datastore=blob_store)
h_time_series_3_pivot_pd = PipelineData("h_time_series_3_pivot",datastore=blob_store)
print("== Intermediate PipelineData references created")

== Intermediate PipelineData references created


### Pipelines steps using datasources and intermediate data
Machine learning pipelines can have many steps and these steps could use or reuse datasources and intermediate data. Here's how we construct such a pipeline:

In [6]:
# The best practice is to use separate folders for scripts and its dependent files
# for each step and specify that folder as the source_directory for the step.
# This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted).
# Since changes in any files in the source_directory would trigger a re-upload of the snapshot, this helps
# keep the reuse of the step when there are no changes in the source_directory of the step.
source_directory_pivot = 'src/pivot'
source_directory_join = 'src/join'
source_directory_groupby_aggregate = 'src/groupby-aggregate'

In [7]:
# PIVOT all hourly time series + normalize timestamps

h_time_series_1_pivot_step = PythonScriptStep(
    script_name="pivot.py", 
    arguments=["--date_column","MYDATE",
               "--hour_column","HOUR",
               "--datetime_column_name","DATETIME",
               "--pivot_columns","NODE_ID",
               "--value_column","MW",
               "--output", h_time_series_1_pivot_pd],
    inputs=[h_time_series_1_ds.as_named_input("time_series")],
    outputs=[h_time_series_1_pivot_pd],
    compute_target=aml_compute, 
    source_directory=source_directory_pivot,
    runconfig=run_config
)
print("== PythonScriptStep time_series_1_pivot_step created")

h_time_series_2_pivot_step = PythonScriptStep(
    script_name="pivot.py", 
    arguments=["--date_column","MYDATE",
               "--hour_column","HOUR",
               "--datetime_column_name","DATETIME",
               "--pivot_columns","NODE_ID",
               "--value_column","MW",
               "--output", h_time_series_2_pivot_pd],
    inputs=[h_time_series_2_ds.as_named_input("time_series")],
    outputs=[h_time_series_2_pivot_pd],
    compute_target=aml_compute, 
    source_directory=source_directory_pivot,
    runconfig=run_config
)
print("== PythonScriptStep time_series_2_pivot_step created")

h_time_series_3_pivot_step = PythonScriptStep(
    script_name="pivot.py", 
    arguments=["--date_column","DATE",
               "--hour_column","HE",
               "--datetime_column_name","DATETIME",
               "--pivot_columns","",
               "--value_column","CLOAD",
               "--output", h_time_series_3_pivot_pd],
    inputs=[h_time_series_3_ds.as_named_input("time_series")],
    outputs=[h_time_series_3_pivot_pd],
    compute_target=aml_compute, 
    source_directory=source_directory_pivot,
    runconfig=run_config
)
print("== PythonScriptStep time_series_3_pivot_step created")

== PythonScriptStep time_series_1_pivot_step created
== PythonScriptStep time_series_2_pivot_step created
== PythonScriptStep time_series_3_pivot_step created


#### Define a Step that consumes intermediate data and produces intermediate data
In this step, we define a step that consumes an intermediate data and produces intermediate data.

**Open `join.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.** 

In [8]:
# Joining all HOURLY pivoted time-series

#h_time_series_joined_ds = PipelineData('h_time_series_joined', datastore=blob_store).as_dataset()
#h_time_series_joined_ds = h_time_series_joined_ds.register(name='h_time_series_joined', create_new_version=True)

h_time_series_joined_pd = PipelineData("h_time_series_joined",datastore=blob_store)

# Join Step
h_join_step = PythonScriptStep(
    script_name="join.py",
    arguments=["--join_column", "DATETIME",
               "--input_1", h_time_series_1_pivot_pd,
               "--input_2", h_time_series_2_pivot_pd,
               "--input_3", h_time_series_3_pivot_pd,
               "--output", h_time_series_joined_pd],
    inputs=[h_time_series_1_pivot_pd,
            h_time_series_2_pivot_pd,
            h_time_series_3_pivot_pd],
    outputs=[h_time_series_joined_pd],
    compute_target=aml_compute, 
    source_directory=source_directory_join,
    runconfig=run_config
)

print("== PythonScriptStep h_join_step created")

== PythonScriptStep h_join_step created


In [9]:
# Generate stats for HOURLY joined series GROUP BY DAILY
#h_time_series_joined_groupby_aggregated_ds = PipelineData('h_time_series_joined_groupby_aggregated', datastore=blob_store).as_dataset()
#h_time_series_joined_groupby_aggregated_ds = h_time_series_joined_groupby_aggregated_ds.register(name='h_time_series_joined_groupby_aggregated', create_new_version=True)

h_time_series_joined_groupby_aggregated_pd = PipelineData("h_time_series_joined_groupby_aggregated",datastore=blob_store)

# groupby-aggregate step
groupby_aggregate_step = PythonScriptStep(
    script_name="groupby-aggregate.py",
    arguments=["--datetime_column", "DATETIME",
               "--date_column_name","RDATE",
               "--input_pd", h_time_series_joined_pd,
               "--output_pd", h_time_series_joined_groupby_aggregated_pd],
    inputs=[h_time_series_joined_pd],
    outputs=[h_time_series_joined_groupby_aggregated_pd],
    compute_target=aml_compute,
    source_directory=source_directory_groupby_aggregate,
    runconfig=run_config
)

print("== PythonScriptStep groupby_aggregate_step created")

== PythonScriptStep groupby_aggregate_step created


In [10]:
# Joining all DAILY time-series
#d_time_series_joined_ds = PipelineData('d_time_series_joined', datastore=blob_store).as_dataset()
#d_time_series_joined_ds = d_time_series_joined_ds.register(name='d_time_series_joined', create_new_version=True)

d_time_series_joined_pd = PipelineData("d_time_series_joined",datastore=blob_store)

# Join Step
d_join_step = PythonScriptStep(
    script_name="join.py",
    arguments=["--join_column", "RDATE",
               "--input_1", d_time_series_1_dr,
               "--input_2", d_time_series_2_dr,
               "--output", d_time_series_joined_pd,
               "--cleanup_date_column", "RDATE"],
    inputs=[d_time_series_1_dr,
            d_time_series_2_dr],
    outputs=[d_time_series_joined_pd],
    compute_target=aml_compute,
    source_directory=source_directory_join,
    runconfig=run_config
)

print("== PythonScriptStep d_join_step created")

== PythonScriptStep d_join_step created


In [11]:
# FINAL JOIN TO GET THE USE CASE 1 TRAINING DATA SET READY
# JOIN hourly summarized by day and daily time series
d_use_case_1_pd = PipelineData("d_use_case_1",datastore=blob_store)

# Join Step
use_case_1_join_step = PythonScriptStep(
    script_name="join.py",
    arguments=["--join_column", "RDATE",
               "--input_1", h_time_series_joined_groupby_aggregated_pd,
               "--input_2", d_time_series_joined_pd,
               "--output", d_use_case_1_pd],
    inputs=[h_time_series_joined_groupby_aggregated_pd,
            d_time_series_joined_pd],
    outputs=[d_use_case_1_pd],
    compute_target=aml_compute,
    source_directory=source_directory_join,
    runconfig=run_config
)

print("== PythonScriptStep use_case_1_join_step created")

== PythonScriptStep use_case_1_join_step created


### Build the pipeline and submit an Experiment run

In [12]:
#pipeline = Pipeline(workspace=ws, steps=[h_join_step])
#pipeline = Pipeline(workspace=ws, steps=[groupby_aggregate_step])
#pipeline = Pipeline(workspace=ws, steps=[d_join_step])
pipeline = Pipeline(workspace=ws, steps=[use_case_1_join_step])
print ("== Pipeline is built")

== Pipeline is built


In [13]:
pipeline_run = Experiment(ws, 'use-case-1-data-prep').submit(pipeline)
print("== Pipeline is submitted for execution")

Created step join.py [19e9c3d1][af286a2f-13bd-4b1c-95ab-5c40e116abd3], (This step is eligible to reuse a previous run's output)
Created step groupby-aggregate.py [b596c8e9][e3e57e13-ee0b-4d77-b7fe-ec9b75bf2571], (This step is eligible to reuse a previous run's output)
Created step join.py [ec68d6b8][2da770e4-25b5-4809-89a6-1dff81deda69], (This step is eligible to reuse a previous run's output)
Created step pivot.py [18370d95][9f725572-a551-4fb6-a60c-24a5e723b764], (This step is eligible to reuse a previous run's output)
Created step pivot.py [f83bbd72][2f43d539-a993-442e-8305-905ec44aad72], (This step is eligible to reuse a previous run's output)
Created step pivot.py [cec9d01e][ee04757b-a214-413d-adb5-d1d8bdb635f5], (This step is eligible to reuse a previous run's output)
Created step join.py [10582848][b76ecc78-7aeb-406f-99c4-511a7cf5f5c8], (This step is eligible to reuse a previous run's output)
Using data reference d_time_series_1 for StepId [979a637b][466adead-675c-4485-b1f6-3a082

In [14]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

### Wait for pipeline run to complete

In [15]:
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: aba3354f-4a1b-4c19-a0c7-7990a3249a42
Link to Portal: https://ml.azure.com/experiments/use-case-1-data-prep/runs/aba3354f-4a1b-4c19-a0c7-7990a3249a42?wsid=/subscriptions/c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60/resourcegroups/azure-ml-workshop/workspaces/agd-ml-demo
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: f430bc5d-6473-4c33-9353-5d658808507c
Link to Portal: https://ml.azure.com/experiments/use-case-1-data-prep/runs/f430bc5d-6473-4c33-9353-5d658808507c?wsid=/subscriptions/c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60/resourcegroups/azure-ml-workshop/workspaces/agd-ml-demo

StepRun(pivot.py) Execution Summary
StepRun( pivot.py ) Status: Finished
{'runId': 'f430bc5d-6473-4c33-9353-5d658808507c', 'target': 'agd-training-cpu', 'status': 'Completed', 'startTimeUtc': '2020-04-03T18:20:30.279874Z', 'endTimeUtc': '2020-04-03T18:20:30.386505Z', 'properties': {'azureml.reusedrunid': 'ebcd9334-ff7e-486a-a8f1-a7e3c2d009bd', 'azureml.reusednodeid': 'e030d636', 'azure




StepRunId: 9bb4f1dd-1f6d-467a-94e5-f8bc5a617262
Link to Portal: https://ml.azure.com/experiments/use-case-1-data-prep/runs/9bb4f1dd-1f6d-467a-94e5-f8bc5a617262?wsid=/subscriptions/c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60/resourcegroups/azure-ml-workshop/workspaces/agd-ml-demo

StepRun(join.py) Execution Summary
StepRun( join.py ) Status: Finished
{'runId': '9bb4f1dd-1f6d-467a-94e5-f8bc5a617262', 'target': 'agd-training-cpu', 'status': 'Completed', 'startTimeUtc': '2020-04-03T18:20:30.263195Z', 'endTimeUtc': '2020-04-03T18:20:30.430064Z', 'properties': {'azureml.reusedrunid': 'a43da713-412c-4297-b5c2-07ae5286e4e2', 'azureml.reusednodeid': '557b3526', 'azureml.reusedpipeline': '0ea5c33d-dc8b-47cb-bf33-314eeb965b03', 'azureml.reusedpipelinerunid': '0ea5c33d-dc8b-47cb-bf33-314eeb965b03', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': '10582848', 'ContentSnapshotId': '26933105-d9fb-468a-8d51-46a1eb898b14', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml


StepRun(pivot.py) Execution Summary
StepRun( pivot.py ) Status: Finished
{'runId': '6758af63-6818-48b2-b538-012df36122dc', 'target': 'agd-training-cpu', 'status': 'Completed', 'startTimeUtc': '2020-04-03T18:20:30.256345Z', 'endTimeUtc': '2020-04-03T18:20:30.376958Z', 'properties': {'azureml.reusedrunid': '95520a21-fa7c-4ffd-bf08-a3854febd1c2', 'azureml.reusednodeid': 'fde23b80', 'azureml.reusedpipeline': '26a41e5c-2a07-4d05-8c3d-825a72d02f9c', 'azureml.reusedpipelinerunid': '26a41e5c-2a07-4d05-8c3d-825a72d02f9c', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': '18370d95', 'ContentSnapshotId': '40de9f0c-d8ea-4845-8b5f-b4dc67c79af6', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'aba3354f-4a1b-4c19-a0c7-7990a3249a42', '_azureml.ComputeTargetType': 'amlcompute', 'AzureML.DerivedImageName': 'azureml/azureml_fbecfa6255cba4c1cc15571eceadee24', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/proce




StepRunId: a5c8194a-627a-466d-95a9-f89441801991
Link to Portal: https://ml.azure.com/experiments/use-case-1-data-prep/runs/a5c8194a-627a-466d-95a9-f89441801991?wsid=/subscriptions/c5ec24ce-9c5f-4da2-bf12-9ca8e9758d60/resourcegroups/azure-ml-workshop/workspaces/agd-ml-demo

StepRun(pivot.py) Execution Summary
StepRun( pivot.py ) Status: Finished
{'runId': 'a5c8194a-627a-466d-95a9-f89441801991', 'target': 'agd-training-cpu', 'status': 'Completed', 'startTimeUtc': '2020-04-03T18:20:35.261692Z', 'endTimeUtc': '2020-04-03T18:20:35.346011Z', 'properties': {'azureml.reusedrunid': 'd60a2b61-d4de-4e2f-91a3-6dd585fa365b', 'azureml.reusednodeid': '4596da55', 'azureml.reusedpipeline': '26a41e5c-2a07-4d05-8c3d-825a72d02f9c', 'azureml.reusedpipelinerunid': '26a41e5c-2a07-4d05-8c3d-825a72d02f9c', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': 'f83bbd72', 'ContentSnapshotId': '40de9f0c-d8ea-4845-8b5f-b4dc67c79af6', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azure


StepRun(groupby-aggregate.py) Execution Summary
StepRun( groupby-aggregate.py ) Status: Finished
{'runId': 'a5be812e-a434-4278-8428-796729620feb', 'target': 'agd-training-cpu', 'status': 'Completed', 'startTimeUtc': '2020-04-03T18:20:36.550984Z', 'endTimeUtc': '2020-04-03T18:20:36.683549Z', 'properties': {'azureml.reusedrunid': 'd06254cd-e258-4dcb-8942-5d312cabf671', 'azureml.reusednodeid': 'b799be95', 'azureml.reusedpipeline': '4cea5aba-20aa-4965-8f2d-e1e86a25c4da', 'azureml.reusedpipelinerunid': '4cea5aba-20aa-4965-8f2d-e1e86a25c4da', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': 'b596c8e9', 'ContentSnapshotId': '62441d21-969f-4cb4-86b5-f4c30da1ecbc', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'aba3354f-4a1b-4c19-a0c7-7990a3249a42', '_azureml.ComputeTargetType': 'amlcompute', 'AzureML.DerivedImageName': 'azureml/azureml_fbecfa6255cba4c1cc15571eceadee24', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFi

'Finished'

### See Outputs

See where outputs of each pipeline step are located on your datastore.

***Wait for pipeline run to complete, to make sure all the outputs are ready***

In [16]:
# Get Steps
for step in pipeline_run.get_steps():
    print("== Outputs of step " + step.name)
    
    # Get a dictionary of StepRunOutputs with the output name as the key 
    output_dict = step.get_outputs()
    
    for name, output in output_dict.items():
        output_reference = output.get_port_data_reference() # Get output port data reference
        print("\tname: " + name)
        print("\tdatastore: " + output_reference.datastore_name)
        print("\tpath on datastore: " + output_reference.path_on_datastore)

== Outputs of step join.py
	name: d_use_case_1
	datastore: workspaceblobstore
	path on datastore: azureml/5f26745b-80f1-459a-b27e-52b72d914853/d_use_case_1
== Outputs of step groupby-aggregate.py
	name: h_time_series_joined_groupby_aggregated
	datastore: workspaceblobstore
	path on datastore: azureml/d06254cd-e258-4dcb-8942-5d312cabf671/h_time_series_joined_groupby_aggregated
== Outputs of step join.py
	name: h_time_series_joined
	datastore: workspaceblobstore
	path on datastore: azureml/cb6ac549-7778-4d2b-b71f-3b4376936dd1/h_time_series_joined
== Outputs of step pivot.py
	name: h_time_series_2_pivot
	datastore: workspaceblobstore
	path on datastore: azureml/d60a2b61-d4de-4e2f-91a3-6dd585fa365b/h_time_series_2_pivot
== Outputs of step pivot.py
	name: h_time_series_3_pivot
	datastore: workspaceblobstore
	path on datastore: azureml/ebcd9334-ff7e-486a-a8f1-a7e3c2d009bd/h_time_series_3_pivot
== Outputs of step join.py
	name: d_time_series_joined
	datastore: workspaceblobstore
	path on data

In [17]:
# REGISTER a new version of the final output as a Dataset

from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

# find output dataset
for step in pipeline_run.get_steps():
    output_dict = step.get_outputs()
    for name, output in output_dict.items():
        if name == 'd_use_case_1':
            # generate a Tabular DataSet for it
            datastore_path = [DataPath(blob_store, output_reference.path_on_datastore)]
            ds = Dataset.Tabular.from_delimited_files(datastore_path)
            dataset_name = 'd_use_case_1'
            ds.register(ws, name=dataset_name, create_new_version=True)
            print("== Registered new version of dataset: " + dataset_name)

== Registered new version of dataset: d_use_case_1
